Clone Repo

In [1]:
import os 

# Define the directory where you want to check for the existence of a file
dir_path = 'MachineLearningPortfolio'

if os.path.exists(dir_path):
    os.chdir(dir_path)
    !git pull origin
else:
    !git clone https://github.com/bchenley/MachineLearningPortfolio.git

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), 809 bytes | 202.00 KiB/s, done.
From https://github.com/bchenley/MachineLearningPortfolio
   2946c17..bbffdfc  main       -> origin/main
Updating 2946c17..bbffdfc
Fast-forward
 SentimentAnalysis/src/TextPreprocessor.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


Change working directory to `SentimentAnalysis`

In [2]:
cd SentimentAnalysis/

/Users/brandonhenley/Library/CloudStorage/GoogleDrive-henley.brandon@gmail.com/My Drive/MLProjects/SentimentAnalysis_IMDB/notebooks/MachineLearningPortfolio/SentimentAnalysis


Install requirements

In [ ]:
pip install -r requirements.txt

In [3]:
import os

import numpy as np
import pandas as pd

import sklearn
import torch

import zipfile

Import relevant modules. In this notebook we need `unzip` to unzip the dataset zipfile and `TextPreprocessor` to preprocess all the reviews.

In [4]:
from src import unzip, TextPreprocessor

Unzip File. Change `data_path` to the path to the zipfile saved on your computer. The `IMDB Dataset.csv` will load into the currrent working directory, unless you specificy `sink` in `unzip`.

In [5]:
sink = os.getcwd()
source = '/Users/brandonhenley/Library/CloudStorage/GoogleDrive-henley.brandon@gmail.com/My Drive/MLProjects/SentimentAnalysis_IMDB/data/archive.zip'
unzip(source, sink)


Load data into a DataFrame

In [6]:
imdb = pd.read_csv(os.path.join(sink, 'IMDB Dataset.csv'))

Basic parameters of dataset

In [30]:
total_reviews = imdb.shape[0]

num_positive_reviews = sum(imdb['sentiment'] == 'positive')
num_negative_reviews = sum(imdb['sentiment'] == 'negative')

print(f"Total number of reviews = {total_reviews}.")
print(f"Total number of positive reviews = {num_positive_reviews}.")
print(f"Total number of negative reviews = {num_negative_reviews}.")

Total number of reviews = 50000.
Total number of positive reviews = 25000.
Total number of negative reviews = 25000.


Print a few positive and negative reviews.

In [31]:
idx_4_positive_reviews = np.where(imdb['sentiment'] == 'positive')[0][np.random.choice(num_positive_reviews-1, size=4, replace=False)]
idx_4_negative_reviews = np.where(imdb['sentiment'] == 'negative')[0][np.random.choice(num_negative_reviews-1, size=4, replace=False)]

In [32]:
print(f"Positive Reviews")
for i,review in enumerate(imdb.iloc[idx_4_positive_reviews, 0]):
  print(f"Review {idx_4_positive_reviews[i]}: {review}")


Positive Reviews
Review 37771: I absolutely loved this movie. I bought it as soon as I could find a copy of it. This movie had so much emotion, and felt so real, I could really sympathize with the characters. Every time I watch it, the ending makes me cry. I can really identify with Busy Phillip's character, and how I would feel if the same thing had happened to me.<br /><br />I think that all high schools should show this movie, maybe it will keep people from wanting to do the same thing. I recommend this movie to everybody and anybody. Especially those who have been affected by any school shooting.<br /><br />It truly is one of the greatest movies of all time.
Review 42280: You have GOT to see this movie... I saw it, as a 13 year old, at the theater, on my very first date... Fast forward over 20 years and I'm now gay (Thanks a lot One Dark Night!! LOL!). This movie creeped the hell out of me as a kid (mausoleums still do!), but as an adult, the thrill of this movie isn't in the story

In [33]:
print(f"Negative Reviews")
for i,review in enumerate(imdb.iloc[idx_4_negative_reviews, 0]):
  print(f"Review {idx_4_negative_reviews[i]}: {review}")


Negative Reviews
Review 31529: I saw it on video. Predictable, horrid acting, film flubs. What more can be said, this movie sucks. The actors are annoying to say the least. This was suppose to be a comedy, but there was only one funny moment, other than that is was painful to watch for me.<br /><br />1 out of 10. PASS!
Review 31166: After watching about half of this I was ready to give up and turn it off, but I endured to the end. This is a movie that tries to be a romantic comedy and fails. The acting is poor---much worse than the acting in 80s T&A movies.<br /><br />There are several attempts at humour that fail miserably and the movie is 100% predictable. Perhaps if you are a teenager this movie will hold some appeal, but for those that have seen many movies, you will know how the film turns out after the first 10 minutes. The rest of your time will be spent in agony waiting for the ending credits to roll.<br /><br />Don't waste your time watching this.
Review 20783: ...and that's s

Preprocessing

In [8]:
# Instantiate TextPreprocessor
tp = TextPreprocessor()

In [15]:
imdb

,review,sentiment,review_p
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production the filming tech...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically there is a family where a little boy...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love in the time of money is a ...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,i am going to have to disagree with the previo...


In [14]:
# Fit and transform reviews
tp.fit(imdb['review'].tolist())
review_p = tp.transform(imdb['review'].tolist())

In [16]:
# Creata new DataFrame with labeled preprocessed data
imdb_p = pd.DataFrame({'review': review_p})
imdb_p['sentiment'] = imdb['sentiment']